Burrows-Wheeler-Transform

In [1]:
def bwt(s):
    assert"\003" not in s, "Input string cannot contain ETX characters"
    s = s + "\003"
    table = sorted(s[i:] + s[:i] for i in range(len(s))) 
    print("table done")
    last_column = [row[-1:] for row in table]
    return "".join(last_column)

path = "./repetitions_data.txt"
file1 = open(path,'r')
data = file1.read()

encoded_data = bwt(data)

outpath = "bwt.txt"

outfile = open(outpath,"w")
outfile.write(encoded_data)

print("done")

table done
done


Run Length Encoding

In [1]:
from collections import OrderedDict

def runLengthEncoding(input): 
    dict=OrderedDict.fromkeys(input, 0) 
    for ch in input: 
        dict[ch] += 1
    output = '' 
    for key,value in dict.items(): 
         output = output + key + str(value)+ ":" # for demarcation - Decompression
    return output 

if __name__ == "__main__": 
    path = "./bwt.txt"
    
    file1 = open(path,'r')
    data = file1.read()
    input=data
    output = runLengthEncoding(input) 
    print(output)
    outpath = "./bwt_rle.txt"
    outfile = open(outpath,"w")
    outfile.write(output)
    print("done")

.435:
1455:?80:	39:!75:”45:k585:,695:s2780:o3510:y840:r2085:d1470:t2955:;100:n2215:e3955:35:010:110:-140:u1480: 9887:p775:*110:f710:l1860:m780::5:w860:–55:’85:h2085:g1070:I350:R30:—5:a2655:i2235:x80:q45:z125:A185:ó5:"30:D25:P70:)5:520:“45:T165:O40:S140:﻿1:í5:V5:E10:b450:M35:c1455:F80:Y55:B80:j95:ñ30:C30:ú5:(5:v155:H80:W110:J15:K5:L5:G20:'155:N5:Ñ5:á10:è10:‘5:1:
done


Lempel-Ziv Algorithm

In [3]:
class Lz77:
    def __init__(self, sliding_window_size, search_buffer_size):
        self.sws = sliding_window_size
        self.sbs = search_buffer_size
        self.encoded_text = ""
        self.codes = []
    
    def compressor(self,data):
        encoded_text  = ""
        position = 0 
        offset = 0
        max_match_length = 0
        while(position<len(data)):
            #print("a")
            offset,max_match_length,codeword =  self.match(position,data)
            self.codes.append([offset,max_match_length,codeword])
            position += max_match_length+1
            #print("c", max_match_length)
            encoded_text = encoded_text+str(offset)+" "+ str(max_match_length)+" "+codeword+" "
        self.encoded_text = encoded_text
        return encoded_text

    def match(self, position, data):
        sb = self.sbs
        i = 1
        key = data[position]
        max_match_length = 0
        offset = 0
        while((position-i)>=0 and i!=sb+1):
            #print("b")
            length = 0
            if(data[position-i]==key):
                length+=1
                j = 1
                while(position-i+j!= len(data) and position+j!=len(data)):
                    if(data[position-i+j]== data[position+j]):
                        length+=1
                        j+=1
                    else:
                        break
                if(max_match_length<length):
                    max_match_length = length
                    offset = i
            i +=1
            if(position+max_match_length==len(data)):
                return offset,max_match_length,""
        return offset,max_match_length,data[position+max_match_length]

    def decompressor(self):
        decoded_text = ""
        for code in (self.codes):
            pointer_position = len(decoded_text)
            offset = code[0]
            length = code[1]
            code_word = code[2]
            for j in range(length):
                decoded_text += decoded_text[pointer_position-offset+j] 
            decoded_text+=code_word

        return decoded_text

path = "./binary_data/bwt.txt"
outpath = "./bwt_lz77.txt"
outpath2 = "./lz77_decompressed.txt"
file1 = open(path,'r')
data = file1.read()
sliding_window_size = 13
search_buffer_size = 7
lz = Lz77(sliding_window_size, search_buffer_size)
compressed = lz.compressor(data)
output_compressed = open(outpath,'w')
output_compressed.write(compressed)

35263

Huffman Encoding

In [3]:
import heapq
import pandas as pd 
import numpy as np 
from collections import Counter
import re
from functools import total_ordering

@total_ordering
class Node:
    def __init__(self, freq,char):
        self.left = None
        self.right= None
        self.fre = freq
        self.key = char
    
    # defining comparators less_than and equals
    def __lt__(self, other):
        return self.fre < other.fre

    def __eq__(self,other):
        if(other == None):
            return False
        if(not isinstance(other , Node)):
            return False
        return self.fre == other.fre

class Tree:
    def __init__(self,listfreq):
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}
        for key in listfreq:
            node = Node(listfreq[key], key)
            heapq.heappush(self.heap,node)

        while(len(self.heap)>1):
            t1 = heapq.heappop(self.heap)
            t2 = heapq.heappop(self.heap)
            count = t1.fre + t2.fre
            D = Node(count,None)
            D.left = t1
            D.right = t2
            heapq.heappush(self.heap, D)
	
    
    def huffmancode(self, root, current_code):
        if (root == None) :
            return

        if(root.key != None):
            self.codes[root.key] = current_code
            self.reverse_mapping[current_code] = root.key
            return
        
        self.huffmancode(root.left,current_code+"0")
        self.huffmancode(root.right, current_code+"1")

    def encoded_te(self, text):
        return  "".join(self.codes[let] for let in text)
    def compress(self, text, output_path):
        encoded_text =  "".join(self.codes[let] for let in text)
        extra_padding = 8 - len(encoded_text)%8
        for i in range(extra_padding):
            encoded_text += "0"
        
        padded_count = "{0:08b}".format(extra_padding)

        padded_encoded_text = padded_count + encoded_text

        if(len(padded_encoded_text)%8 !=0):
            print(" Not padded properly")
            exit(0)

        b = bytearray()

        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8]
            b.append(int(byte,2))
        
    
        output = open(output_path, 'ab')

        output.write(bytes(b))
        output.close()
        print("Compressed Successfully")
        return output_path
        

def main():

    path = "./bwt_rle.txt"
    output_path = "./bwt_rle_huffman.bin"
    file1 = open(path,'r')
    data = file1.read()
    res = Counter(data)
    tree =  Tree(res)
    root = heapq.heappop(tree.heap)
    s = ''
    tree.huffmancode(root,s)
    #print(tree.codes)
    compressed_path = tree.compress(data,output_path)

if __name__ == "__main__":
    main()


Compressed Successfully


In [62]:
def deflate():
    path = "./test_data.txt"
    output_path = "./entire40.bin"
    output_path2 = "./deflate_compressed65.bin"
    file1 = open(path,'r')
    data = file1.read()
    block_size = 1024*65

    sliding_window_size = 4096*4/3
    search_buffer_size = sliding_window_size*3/4

    lz = Lz77(sliding_window_size, search_buffer_size)
    lz_compressed = lz.compressor(data)
    
    res = Counter(lz_compressed)
    huffman_tree =  Tree(res)
    root = heapq.heappop(huffman_tree.heap)
    s = ''
    huffman_tree.huffmancode(root,s)
    
    '''entire_array = bytearray()

    entire_array = huffman_tree.compress(lz_compressed)

    output = open(output_path, 'wb')

    output.write(bytes(entire_array))

    print("Compressed Successfully")'''

    #final = huffman_tree.compress(lz_compressed,output_path)
    
    i=0
    encoded_text = ''
    while(i<len(data)):
        block_data = data[i:i+block_size]
        i+= block_size

        lz_block = Lz77(sliding_window_size, search_buffer_size)
        lz_block_compressed = lz_block.compressor(block_data)

        res_block = Counter(lz_block_compressed)
        hblock_tree = Tree(res_block)
        root_block = heapq.heappop(hblock_tree.heap)
        s_block = ''
        hblock_tree.huffmancode(root_block, s_block)
        etext_block = hblock_tree.encoded_te(lz_block_compressed) 
        etext = huffman_tree.encoded_te(lz_block_compressed)

        if( len(etext)<len(etext_block)) :
            huffman_tree.compress(lz_block_compressed,output_path2)
            print("a")
        else :
            hblock_tree.compress(lz_block_compressed,output_path2)
            print("b")

deflate()

print("done")

Compressed Successfully
b
Compressed Successfully
b
done
